In [1]:
import os
import random
import numpy as np
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

from multiprocessing import cpu_count
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.nn import CrossEntropyLoss
from torchvision.datasets import CIFAR10

In [2]:
seed = 42

os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

trainset = CIFAR10(root='./cifar10', train=True, transform=transform, download=True)
testset = CIFAR10(root='./cifar10', train=False, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10


In [22]:
# Train set, validation set split
train_idx, valid_idx = train_test_split(np.arange(len(trainset)), test_size=0.1, random_state=42, shuffle=True, stratify=trainset.targets)

batch_size = 100
num_workers = int(cpu_count() / 2)

train_loader = DataLoader(trainset, batch_size=batch_size, sampler=SubsetRandomSampler(train_idx), num_workers=num_workers)
valid_loader = DataLoader(trainset, batch_size=batch_size, sampler=SubsetRandomSampler(valid_idx), num_workers=num_workers)
test_loader = DataLoader(testset, batch_size=batch_size, num_workers=num_workers)

print(len(train_loader))
print(len(valid_loader))

450
50


In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
class Block(nn.Module):
    def __init__(self, in_ch, _is_firstblock):
        super(Block, self).__init__()
        self._is_firstblock = _is_firstblock
        if _is_firstblock:
            self.BN1 = nn.BatchNorm2d(in_ch)
            self.Conv1 = nn.Conv2d(in_ch, in_ch*2, 3, stride=2, padding=1)
            self.skip_Conv = nn.Conv2d(in_ch, in_ch*2, 1, stride=2)
            self.BN2 = nn.BatchNorm2d(in_ch*2)
            self.Conv2 = nn.Conv2d(in_ch*2, in_ch*2, 3, stride=1, padding=1)
        else:
            self.BN1 = nn.BatchNorm2d(in_ch)
            self.Conv1 = nn.Conv2d(in_ch, in_ch, 3, stride=1, padding=1)
            self.BN2 = nn.BatchNorm2d(in_ch)
            self.Conv2 = nn.Conv2d(in_ch, in_ch, 3, stride=1, padding=1)
        self.Relu = nn.ReLU()
    
    def forward(self, x):
        if not self._is_firstblock:
            skip = x
        first_batchnorm = self.BN1(x)
        first_relu = self.Relu(first_batchnorm)
        if self._is_firstblock:
            skip = self.skip_Conv(first_relu)
        first_conv = self.Conv1(first_relu)
        sec_batchnorm = self.BN2(first_conv)
        sec_relu = self.Relu(sec_batchnorm)
        sec_conv = self.Conv2(sec_relu)
        out = sec_conv+skip
        return out



class ResNet(nn.Module):
    def __init__(self, nblk_stage1, nblk_stage2, nblk_stage3, nblk_stage4):
        super(ResNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, 3,stride=1, padding=1)
        modules = []
        for i in range(nblk_stage1):
            modules.append(Block(64, False))
        for i in range(nblk_stage2):
            if i == 0:
                modules.append(Block(64, True))
            else:
                modules.append(Block(128, False))
        for i in range(nblk_stage3):
            if i == 0:
                modules.append(Block(128, True))
            else:
                modules.append(Block(256, False))
        for i in range(nblk_stage4):
            if i == 0:
                modules.append(Block(256, True))
            else:
                modules.append(Block(512, False))
        self.wholeBlock = nn.Sequential(*modules)
        self.fc = nn.Linear(512,10)

    def forward(self, x):
        first_conv = self.conv1(x)
        after_stages = self.wholeBlock(first_conv)
        last_pool =F.avg_pool2d(after_stages,4,stride=4)
        out = self.fc(last_pool.permute(0,2,3,1))
        return torch.squeeze(out)

In [25]:
net = ResNet(nblk_stage1=3, nblk_stage2=3,
                     nblk_stage3=3, nblk_stage4=3)
net = net.to(device)

criterion = CrossEntropyLoss()
optimizer = optim.SGD(params=net.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=3, verbose=True)

In [26]:
train_total = len(train_idx)
valid_total = len(valid_idx)

train_batches = len(train_loader)
valid_batches = len(valid_loader)

best_valid_loss = 1024
patience = 0

In [27]:
epochs = 20

for epoch in range(epochs):
    # Train
    net.train()
    
    train_loss = 0
    train_correct = 0
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = net(x)
        loss = criterion(outputs, y)
        
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(y).sum().item()
        
    train_loss = train_loss / train_batches
    train_acc = train_correct / train_total
    
    # Validate
    net.eval()
    
    valid_loss = 0
    valid_correct = 0
    
    with torch.no_grad():
        for x, y in valid_loader:
            x = x.to(device)
            y = y.to(device)
            outputs = net(x)
            loss = criterion(outputs, y)
            
            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            valid_correct += predicted.eq(y).sum().item()
            
    valid_loss = valid_loss / valid_batches
    valid_acc = valid_correct / valid_total
    
    # Save best model
    if best_valid_loss > valid_loss:
        torch.save(net.state_dict(), './2016311076_이동혁.pt')
        best_valid_loss = valid_loss
        patience = 0
        
 
    if (epoch+1) % (epochs/10) == 0:
        print('Epoch: ', epoch + 1)
        print('Loss: ', 'Train %.3f / Valid %.3f' % (train_loss, valid_loss))
        print('Accuracy: ', 'Train %.2f / Valid %.2f' % (train_acc * 100, valid_acc * 100))
        print("[%d/%d] TrainLoss: %.3f, ValLoss: %.3f | TrainAcc: %.2f, ValAcc: %.2f’" % (epoch, epochs, train_loss, valid_loss, train_acc, valid_acc))
    
    scheduler.step(metrics=valid_loss)
    
    # Break training loop if no improvement for 5 consecutive epochs
    if patience == 2:
        break
        
    patience += 1

Epoch:  2
Loss:  Train 0.775 / Valid 0.716
Accuracy:  Train 72.83 / Valid 75.36
[1/20] TrainLoss: 0.775, ValLoss: 0.716 | TrainAcc: 0.73, ValAcc: 0.75’
Epoch:  4
Loss:  Train 0.427 / Valid 0.733
Accuracy:  Train 85.32 / Valid 76.10
[3/20] TrainLoss: 0.427, ValLoss: 0.733 | TrainAcc: 0.85, ValAcc: 0.76’


In [28]:
net.load_state_dict(torch.load('./2016311076_이동혁.pt'))

<All keys matched successfully>

In [29]:
net.eval()

test_loss = 0
test_correct = 0

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = net(x)
        loss = criterion(outputs, y)
        
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        test_correct += predicted.eq(y).sum().item()
        
        if i == 0:
            test_preds = predicted
        else:
            test_preds = torch.cat((test_preds, predicted), dim=0)
            
test_preds = test_preds.cpu()
test_acc = test_correct/len(testset)

print('TestAcc: %.2f' % (test_acc))


TestAcc: 0.77
